In [1]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name

def remove_accents(text):
    return unidecode(text)


def get_country_code(country):
    matching_rows = countries_codes.loc[countries_codes['Country'] == country, ' Code']
    return matching_rows.iloc[0] if not matching_rows.empty else None

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

from fuzzywuzzy import process

#PLAYER DATA - MARKET VALUE AND SALARY
leagues_salary = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1//CSVs we use often/latest_capology_data_money_fixed.csv')

leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/MarketValuesComplete.csv')


leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')

#COUNTRY CODES
countries_codes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/countries_and_codes.csv')

#FIFA RANKINGS
fifa_rankings = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/fifa_ranking-2023-10-26 - fifa_ranking-2023-07-20.csv')

#MATCH DATA - LINEUPS AND RESULTS
# lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_march_24.csv')
lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_april_29.csv')
results_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_results_data_march_24.csv')

/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_49517/1460689172.py:64: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')


In [2]:
leagues_value['Accents Removed'] = leagues_value['Name'].apply(lambda x: unidecode(x))
leagues_value['Team 1 Code'] = leagues_value['Nationality'].apply(get_country_code)

In [3]:
combined_df = pd.read_csv('combined_df_may_7_newest.csv')
for index, row in combined_df.iterrows():
    if row['Status'] == 'SUCCESS' and row['Match Case'] == 'none':
        combined_df.at[index, 'Match Case'] = 'single'

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_49517/2939037280.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv('combined_df_may_7_newest.csv')


In [4]:
combined_df[~combined_df['Competition'].str.startswith('U-20')]

,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
0,0,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Anthony Moris,400000.0,False,working,False,Anthony Moris,A Moris,single
1,1,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Laurent Jans,1200000.0,False,working,False,Laurent Jans,L Jans,single
2,2,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Kevin Malget,150000.0,False,working,False,Kevin Malget,K Malget,single
3,3,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Mario Mutsch,125000.0,False,working,False,Mario Mutsch,M Mutsch,single
4,4,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Chris Philipps,1250000.0,False,working,False,Chris Philipps,C Philipps,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89459,89459,1381,"Olympics, 2020",Brazil vs Ivory Coast,2021-07-25,Brazil,BR,2021,SUCCESS,"['Paulinho', 'Marquinhos', 'Fabinho']",11000000.0,False,Found in Curr Season,False,Claudinho,Claudinho,single
89460,89460,1383,"Olympics, 2020",Saudi Arabia vs Brazil,2021-07-28,Brazil,BR,2021,SUCCESS,"['Paulinho', 'Marquinhos', 'Fabinho']",11000000.0,False,Found in Curr Season,False,Claudinho,Claudinho,single
89461,89461,1387,"Olympics, 2020",Brazil vs Egypt,2021-07-31,Brazil,BR,2021,SUCCESS,"['Paulinho', 'Marquinhos', 'Fabinho']",11000000.0,False,Found in Curr Season,False,Claudinho,Claudinho,single
89462,89462,1389,"Olympics, 2020",Mexico vs Brazil,2021-08-03,Brazil,BR,2021,SUCCESS,"['Paulinho', 'Marquinhos', 'Fabinho']",11000000.0,False,Found in Curr Season,False,Claudinho,Claudinho,single


In [5]:
def add_correct_money(money_column):
    if money_column == 0:
        return 0

    if 'm' in money_column:
        value_rough = money_column.split('m')[0].split('€')[1]
        millinum = (float(value_rough) * 1000000)
        return millinum
    elif 'k' in money_column:
        #print(i, 'k')
        value_rough = money_column.split('k')[0].split('€')[1]
        millinum = (float(value_rough) * 1000)

        return millinum
    elif '-' in money_column:
        return 0
    else:
        return 0


def in_season_around(player_id, curr_season, age):
    if age == "Not Listed": age = 30
    age = int(age)
    year_before = int(curr_season) -1
    year_after = int(curr_season) + 1
    before_season = leagues_value[(leagues_value['PlayerID'] == player_id) & (leagues_value['Season'] == year_before)] #leagues_value_large #leagues_value
    after_season =  leagues_value[(leagues_value['PlayerID'] == player_id) & (leagues_value['Season'] == year_after)]
    if before_season.empty and after_season.empty:
        return -1, "Both Empty"
    if not before_season.empty and after_season.empty:
        if before_season.iloc[0]['Market Value'] != "-":
            if before_season.iloc[0]['Market Value'] == 0:
                return 0, "Only Before + Zero Before"
            else:
                if age >= 30:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * .8), "Only Before" #add_correct_money
                else:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * 1.2), "Only Before" #add_correct_money
        else:
            return 0, "Only Before X"
    if not after_season.empty and before_season.empty:
        
        if after_season.iloc[0]['Market Value'] != "-":
            if after_season.iloc[0]['Market Value'] == 0:
                return 0, "Only After + Zero After"
            else:
                if age <= 30:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * .8), "Only After"
                else:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * 1.2), "Only After"
        else:
            return 0, "Only After X"
    if not before_season.empty and not after_season.empty:
        #print('hi')
        before_season_val = before_season.iloc[0]['Market Value']
        after_season_val = after_season.iloc[0]['Market Value']
        #print(before_season_val, after_season_val)
        if before_season_val != "-" and after_season_val != "-":
            # if type(before_season_val) == str:
            #     before_season_val = add_correct_money(before_season_val)
            # if type(after_season_val) == str:
            #     after_season_val = add_correct_money(after_season_val)
            return ((float(add_correct_money(before_season.iloc[0]['Market Value'])) + float(add_correct_money(after_season.iloc[0]['Market Value'])) / 2)), "Used Average of Season Before and After"
        elif before_season_val != "-" and after_season_val == "-":
            if age >= 30:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * .8)), "Used Season Before * .8 But Had Both"
            else:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * 1.2)), "Used Season Before * 1.2 But Had Both"
        elif before_season_val == "-" and after_season_val != "-":
            if age <= 30:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * .8)), "Used Season After * .8 But Had Both"
            else:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * 1.2)), "Used Season After * 1.2 But Had Both"
        else:
            return 0, "Had Both but Both Were Empty"

def find_in_transfermarkt(player_name, season, country_code, database_name):
    #print(player_name, country_code, season)
    row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == season) & (database_name['Team 1 Code'] == country_code)]
    one_up_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)+1) & (database_name['Team 1 Code'] == country_code)]
    one_down_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)-1) & (database_name['Team 1 Code'] == country_code)]
    print(len(row), len(one_down_row), len(one_up_row))
    final_value = 0
    if not row.empty:
        #print('yeeea')
        player_id = row.iloc[0]['PlayerID']
        age = row.iloc[0]['Age']
        marketval = row.iloc[0]['Market Value']
        if type(marketval) == str:
            #print('der')
            if marketval == '-':
                marketval = 0
            elif '€' in marketval:
                marketval = add_correct_money(marketval)
            marketval = float((marketval)) #add_correct_money
        if marketval != 0:
            #print('wooo')
            #print('***', player_name, marketval)
            return marketval, "Found in Curr Season"
        else:
            #print(marketval)
            new_mv, reason = in_season_around(player_id, season, age)
            print(player_name, new_mv, reason)
            if new_mv == -1 or new_mv == 0 or new_mv == "-":
                return 0, reason
            else:
                return new_mv, reason
    elif not one_up_row.empty:
        player_id = one_up_row.iloc[0]['PlayerID']
        age = one_up_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    elif not one_down_row.empty:
        player_id = one_down_row.iloc[0]['PlayerID']
        age = one_down_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    else:
        return 0, "Did Not Find in Curr Season, One Up, or One Down"
    
def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''

def vet_group_using_initials(jersey, names_found, names_same_nation_last_name):
    correct_names_list = []

    jersey_tokens_initials = [token[0] for token in re.split(r'\s+|[-–]', jersey)]

    names_found_tokens_initials = [token[0] for token in re.split(r'\s+|[-–]', names_found)]

    for name in names_same_nation_last_name:
        name_initials = [token[0] for token in re.split(r'\s+|[-–]', name)]
        
        if set(jersey_tokens_initials) == set(name_initials):
            correct_names_list.append(name)
        elif set(jersey_tokens_initials).issubset(set(name_initials)):
            correct_names_list.append(name)
        #else:
            #print('fail')

    if set(jersey_tokens_initials) == set(names_found_tokens_initials):
        correct_names_list.append(names_found)
    elif set(jersey_tokens_initials).issubset(set(names_found_tokens_initials)):
            correct_names_list.append(names_found)


    return correct_names_list


from fuzzywuzzy import process

def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}

## Name Confusion

In [60]:
name_confusion = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Red Groups/name_confusion.csv')
name_confusion = name_confusion.drop(columns={'Unnamed: 0.1', 'Unnamed: 0'})


In [134]:
done_cases = 0
correct_cases = 0
not_working_cases = 0
for index, row in name_confusion.iterrows():
    if row['Status New'] != 'Done':

        match = row['Match']
        date = row['Date']
        jersey = row['ORIGINAL JERSEY']
        if jersey == 'Hosseini':
            name_confusion.at[index, 'Name(s) Found'] = 'Majid Hosseini'
            name_confusion.at[index, 'ORIGINAL JERSEY'] = 'Majid Hosseini'
            jersey = 'Majid Hosseini'
        
        full_num = row['Season']
        country_code_this = row['Country Code']

        name_confusion_matching_rows = name_confusion[(name_confusion['Match'] == match) & (name_confusion['Date'] == date) & (name_confusion['ORIGINAL JERSEY'] == jersey)]

        #search within dataset
        if len(name_confusion_matching_rows) >= 2:
            print(index, match, date, jersey, row['Name(s) Found'])
            name_confusion.at[index, 'Status New'] = 'Done'
        else:
            not_working_cases += 1
            #Search within the combined DF, in case there's another row with one of the names 
            if 'john' != 'john':
                list_of_names = eval(row['Name(s) Found'])
                for name in list_of_names:
                    name_confusion_matching_rows_big_df = combined_df[(combined_df['Match'] == match) & (combined_df['Date'] == date) & (combined_df['Name(s) Found'] == name)]
                            ame_confusion_matching_rows_big_df) >= 1:
                        list_of_names.remove(name)
                        if len(list_of_names) == 1:
                            
                            correct_name = list_of_names[0]
                            print(f"{index}. original was {row['Name(s) Found']} now it's {correct_name}. found {name}. jersey says {jersey}")
                                
                            number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)

                            if correct_name == 'Mohamed Zaabia':
                                print('yeh')
                                number = 0
                                reason = 'Had both but both were 0'
                            #print(player_this, reason, add_correct_money(number))
                            #add a new row to lookup dataset 1 
                            if type(number) == float:
                                0==0
                            else:
                                number = add_correct_money(number)
                            
                            if reason != "Did Not Find in Curr Season, One Up, or One Down":
                                print('**')
                                print(index, jersey, correct_name, number, reason)
                            #if reason == 'Found in Curr Season':
                                name_confusion.at[index, 'Name(s) Found'] = correct_name
                                name_confusion.at[index, 'Status New'] = 'Done'
                                name_confusion.at[index, 'Market Value New'] = number
                                name_confusion.at[index, 'Match Case'] = 'single'
                                correct_cases += 1
                            else:
                                print('not working', index, number, reason, correct_name, full_num)
                                
                        else:
                            print(index, 'wtf')
                        #print(index, row['Name(s) Found'], jersey, match, date, name)
                    else:
                        0==0
            else:
                
                if jersey == 'Majid Hosseini':
                    list_of_names = 'Majid Hosseini'
                elif jersey == 'Al-Dawsari':
                    list_of_names = 'Salem Al-Dawsari'
                else:
                    list_of_names = eval(row['Name(s) Found'])
                #print('hey', index, match, date, jersey, list_of_names)
                if type(list_of_names) == str:
                    0==0
                    print(list_of_names)
                    correct_name = list_of_names
                    number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                    if type(number) == float:
                        0==0
                    else:
                        number = add_correct_money(number)
                    
                    if reason != "Did Not Find in Curr Season, One Up, or One Down":
                        print('**')
                        print(index, jersey, correct_name, number, reason)
                    #if reason == 'Found in Curr Season':
                        name_confusion.at[index, 'Name(s) Found'] = correct_name
                        name_confusion.at[index, 'Status New'] = 'Done'
                        name_confusion.at[index, 'Market Value New'] = number
                        name_confusion.at[index, 'Match Case'] = 'single'
                        correct_cases += 1
                    else:
                        print('not working', index, number, reason, correct_name, full_num)

                    #print('holy shit', combined_df[combined_df['ORIGINAL JERSEY'] == jersey]['Name(s) Found'].unique())
                else:
                    #if there's only one name in the dataset, you can assume it's that name
                    # existing_data_names_found_this_jersey = combined_df[combined_df['ORIGINAL JERSEY'] == jersey]['Name(s) Found'].unique()
                    # if len(existing_data_names_found_this_jersey) == 1:
                    #     print(index, 'one guy in dataset for this jersey', jersey, existing_data_names_found_this_jersey, list_of_names)
                    #     filtered_names_match_date = filter_using_date_of_match(list_of_names, row['Nationality'], date, False)[1]
                    #     print('filtered to', filtered_names_match_date)
                    # else:
                    #     print(index, 'more than one guy in dataset for this jersey', jersey, existing_data_names_found_this_jersey)
                    #     filtered_names_match_date = filter_using_date_of_match(list_of_names, row['Nationality'], date, False)[1]
                    #     print('filtered to', filtered_names_match_date)
                    #vetted_list_using_initials = vet_group_using_initials(jersey, list_of_names[0], list_of_names)
                    # if player_this in vetted_list_using_initials:
                    #     vetted_list_using_initials.remove(player_this)
                    print(index, list_of_names, jersey)
    else:
        done_cases += 1


#the match date shit isnt working. i think what u need to do is look for existing matches. if they dont have any you might just have to look it up.

#a lot of these guys are cases where the last name of the other person is a fucking sub. ugh 

done_cases, correct_cases, not_working_cases

2 ['Charalampos Kyriakou', 'Vangelis Kyriakou'] Cham Kyriakou
3 ['Daniel James', 'Jordan James'] James
4 ['Daniel James', 'Jordan James'] James
5 ['Daniel James', 'Jordan James'] James
6 ['Ricardo Silva', 'Ricardo Gomes'] Ricardo
7 ['Martin Büchel', 'Marcel Büchel'] M Büchel
8 ['Danel Sinani', 'Dejvid Sinani'] Sinani
9 ['Kyle Casciaro', 'Lee Casciaro'] Casciaro
10 ['Martin Büchel', 'Marcel Büchel'] M Büchel
13 ['Ben Davies', 'Adam Davies'] Davies
14 ['Kyle Casciaro', 'Lee Casciaro'] Casciaro
15 ['Javier Orozco', 'Jonathan Orozco'] J Orozco
16 ['Javier Orozco', 'Jonathan Orozco'] J Orozco
17 ['Javier Orozco', 'Jonathan Orozco'] J Orozco
19 ['Tonia Tisdell', 'Terrence Tisdell'] T Tisdell
20 ['Edson Álvarez', 'Efraín Álvarez'] E Álvarez
21 ['Edson Álvarez', 'Efraín Álvarez'] E Álvarez
22 ['Luuk de Jong', 'Frenkie de Jong'] de Jong
23 ['Luuk de Jong', 'Frenkie de Jong'] de Jong
24 ['Luuk de Jong', 'Frenkie de Jong'] de Jong
25 ['Luuk de Jong', 'Frenkie de Jong'] de Jong
26 ['Luuk de Jong',

(17, 0, 94)

In [136]:
name_confusion.to_csv('name_confusion_edited.csv')

### Name confusion - finishing it

In [67]:
name_confusion = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Red Groups/ADD TO COMBINED DF/name_confusion_edited.csv')

In [73]:
for index, row in name_confusion.iterrows():
    if row['Match Case'] == 'single' and row['Status New'] != 'Done':
        print(index)
        name = row['Name(s) Found']
        full_num = row['Season']
        country_code_this = row['Country Code']
        number, reason = find_in_transfermarkt(name, full_num, country_code_this, leagues_value)
        print(index, name, full_num, number, reason)
        if reason != 'Did Not Find in Curr Season, One Up, or One Down':
            name_confusion.at[index, 'Status New'] = 'Done'
            name_confusion.at[index, 'Lookup Return Case'] = reason
            name_confusion.at[index, 'Market Value New'] = number
            #correct_cases += 1

67
0 0 0
67 Lochoshvili 2023 0 Did Not Find in Curr Season, One Up, or One Down


In [74]:
name_confusion.to_csv('name_confusion_edited.csv')

## other_cases_wrong (DONE)

In [6]:
other_cases = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Red Groups/other_cases_wrong_edited.csv')

In [215]:
# for index, row in other_cases.iterrows():
#     if row['Nationality'] == 'Brunei':
#         other_cases.at[index, 'Nationality'] = 'Brunei Darussalam'
#     if row['Nationality'] == 'China PR':
#         other_cases.at[index, 'Nationality'] = 'China'
#     if row['Nationality'] == 'French Guyana':
#         other_cases.at[index, 'Nationality'] = 'French Guiana'
#     if row['Nationality'] == 'Congo DR':
#         other_cases.at[index, 'Nationality'] = 'Democratic Republic of the Congo'

In [75]:
cases = 0
correct_cases = 0
done_cases = 0
for index, row in other_cases.iterrows():
    if row['Nationality'] in ['Brunei Darussalam', 'China', 'Democratic Republic of the Congo']:
        if row['Status New'] != 'Done':
            cases += 1
            match = row['Match']
            date = row['Date']
            jersey = row['ORIGINAL JERSEY']
            nationality = row['Nationality']

            full_num = row['Season']
            country_code_this = row['Country Code']
            dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique() 
            if len(dataset_nationality) <= 1:
                print(index, nationality)
            else:    
                if jersey == 'Ai Kesen':
                    jersey = 'Elkeson'
                elif jersey == 'CZeng':
                    jersey = 'Cheng Zeng'
                elif jersey == 'Jiang Guangtai':
                    jersey = 'Tyias Browning'
                elif jersey == 'J Kasusula':
                    jersey = 'Kilitcho Kasusula'
                elif jersey == 'M Amale':
                    jersey = 'Dieumerci Amale'
                elif jersey == 'N Idumba':
                    jersey = 'Idumba Fasika'
                elif jersey == 'F Hassan':
                    jersey = 'Awangku Fakharazzi'
                elif jersey == 'Li Ke':
                    jersey = 'Nico Yennaris'
                    other_cases.at[index, 'Name(s) Found'] = jersey
                    other_cases.at[index, 'Status New'] = 'Done'
                    other_cases.at[index, 'Market Value New'] = 300000
                    other_cases.at[index, 'Match Case'] = 'single'
                elif jersey == 'H Hamzah':
                    jersey = 'Ahmad Hazwan'
                elif jersey == 'B Bompunga':
                    jersey = 'Bopunga Botuli'
                elif jersey == 'M Ndonga':
                    jersey = 'Hervé Ndonga'
                    other_cases.at[index, 'Name(s) Found'] = jersey
                    other_cases.at[index, 'Status New'] = 'Done'
                    other_cases.at[index, 'Market Value New'] = 50000
                    other_cases.at[index, 'Match Case'] = 'single'
                elif jersey == 'E Bokanga':
                    jersey = 'Eric Bokanga'
                    other_cases.at[index, 'Name(s) Found'] = jersey
                    other_cases.at[index, 'Status New'] = 'Done'
                    other_cases.at[index, 'Market Value New'] = 600000
                    other_cases.at[index, 'Match Case'] = 'single'
                elif jersey == 'L Mbele':
                    jersey = 'Lelo Mbele'
                    other_cases.at[index, 'Name(s) Found'] = jersey
                    other_cases.at[index, 'Status New'] = 'Done'
                    other_cases.at[index, 'Market Value New'] = 100000
                    other_cases.at[index, 'Match Case'] = 'single'
                    
                    correct_cases += 1
                best_match = threshold_player_match(jersey, dataset_nationality)
                if best_match[1] >= 80:
                    vetted_group = vet_group_using_initials(jersey, jersey, best_match[0])
                    if jersey in vetted_group:
                        vetted_group.remove(jersey)
                        
                    if len(vetted_group) == 1:
                        print(index, full_num, jersey, country_code_this, vetted_group)
                        
                        correct_name = vetted_group[0]

                        number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                        print(index, full_num, number, reason)
                        if reason != 'Did Not Find in Curr Season, One Up, or One Down':
                            other_cases.at[index, 'Name(s) Found'] = correct_name
                            other_cases.at[index, 'Status New'] = 'Done'
                            other_cases.at[index, 'Market Value New'] = number
                            other_cases.at[index, 'Match Case'] = 'single'
                            correct_cases += 1
                            
                        #fix the function 
                    elif len(vetted_group) == 2:
                        
                        if jersey == 'Zheng Zhi':
                            vetted_group = ['Zhi Zheng']
                        elif jersey == 'Zheng Long':
                            vetted_group = ['Long Zheng']
                        elif jersey == 'Zou Zheng':
                            vetted_group = ['Zheng Zou']
                        print(index, full_num, jersey, country_code_this, vetted_group)
                        correct_name = vetted_group[0]

                        number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                        print(index, full_num, number, reason)
                        if reason != 'Did Not Find in Curr Season, One Up, or One Down':
                            other_cases.at[index, 'Name(s) Found'] = correct_name
                            other_cases.at[index, 'Status New'] = 'Done'
                            other_cases.at[index, 'Market Value New'] = number
                            other_cases.at[index, 'Match Case'] = 'single'
                            correct_cases += 1
                    else:
                        correct_cases += 1
                        print(index, full_num, jersey, country_code_this, vetted_group, best_match, row['Match'], row['Date'])
                        if best_match[1] == 89:
                            correct_name = best_match[0][0]
                            number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                            print(index, full_num, number, reason)
                            if reason != 'Did Not Find in Curr Season, One Up, or One Down':
                                other_cases.at[index, 'Name(s) Found'] = correct_name
                                other_cases.at[index, 'Status New'] = 'Done'
                                other_cases.at[index, 'Market Value New'] = number
                                other_cases.at[index, 'Match Case'] = 'single'
                                correct_cases += 1

                        

                else:
                    correct_cases += 1
                    print(index, full_num, jersey, country_code_this, best_match, row['Match'], row['Date'])

        else:
            done_cases += 1
    elif row['Nationality'] in ['Martinique', 'French Guiana', 'Guadeloupe', 'Eritrea']:
        other_cases.at[index, 'Status'] = 'FAIL - NO DATA'
        other_cases.at[index, 'Status New'] = 'Done'
        other_cases.at[index, 'Match Case'] = 'none'
        done_cases += 1

    #
cases, correct_cases, done_cases

(0, 0, 365)

In [60]:
#other_cases = other_cases.drop(columns={'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'})

In [62]:
#other_cases.to_csv('other_cases_edited_may_8.csv')

In [79]:
other_cases = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Red Groups/other_cases_edited_may_8.csv')
other_cases = other_cases.drop(columns={'Unnamed: 0'})

combined_df = load_csv_dataset('combined_df_may_8_new.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_49517/1460689172.py:54: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [83]:
for index, row in combined_df.iterrows():
    if combined_df.at[index, 'Name(s) Found'].startswith('['):
        edited_name = eval(combined_df.at[index, 'Name(s) Found'])[0]
        combined_df.at[index, 'Name(s) Found'] = edited_name
        #print(index, combined_df.at[index, 'Name(s) Found'])

combined_df[combined_df['Name(s) Found'].str.startswith('[')]

,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case


In [209]:
correct_cases = 0
cases = 0
done_cases = 0
for index, row in other_cases.iterrows():
    name = row['ORIGINAL JERSEY']
    nationality = row['Nationality']
    full_num = row['Season']
    country_code_this = row['Country Code']
    if row['Status New'] != 'Done':
        matches_big_df = combined_df[(combined_df['ORIGINAL JERSEY'] == name) & (combined_df['Nationality'] == nationality)]['Name(s) Found'].unique()
        if len(matches_big_df) == 1:
            print(index, name, matches_big_df)
            correct_name = matches_big_df[0]
            number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
            print(index, full_num, number, reason)
            if reason != 'Did Not Find in Curr Season, One Up, or One Down':
                other_cases.at[index, 'Name(s) Found'] = correct_name
                other_cases.at[index, 'Status New'] = 'Done'
                other_cases.at[index, 'Market Value New'] = number
                other_cases.at[index, 'Match Case'] = 'single'
                correct_cases += 1
        else:
            cases += 1
            if row['ORIGINAL JERSEY'] == 'Tecatito':
                name = 'Jesús Corona'
            elif row['ORIGINAL JERSEY'] == 'Natkho':
                name = 'Bibras Natcho'
            elif row['ORIGINAL JERSEY'] == 'Hahgverdi':
                name = 'Höccat Haqverdi'
            elif row['ORIGINAL JERSEY'] == 'W Ballakow':
                name = 'Velmyrat Ballakov'
            elif row['ORIGINAL JERSEY'] == 'H Aliqulov':
                name = 'Khusniddin Alikulov'
            elif row['ORIGINAL JERSEY'] == 'H Tn Tài':
                name = 'Tan Tai Ho'
            elif row['ORIGINAL JERSEY'] == 'Nasser Garba':
                name = 'Abdoul Garba'
            elif row['ORIGINAL JERSEY'] == 'A Radosavljevič':
                name = 'Aleksandar Radosavljevic'
            elif row['ORIGINAL JERSEY'] == 'Karčemarskas':
                name = 'Zydrunas Karcemarskas'
            elif row['ORIGINAL JERSEY'] == 'M Islam Nazir':
                name = 'Nasirul Islam'
            elif row['ORIGINAL JERSEY'] == 'Fernández Toro':
                name = 'Roberto Fernández'
            elif row['ORIGINAL JERSEY'] == 'Yerokhin':
                name = 'Aleksandr Erokhin'
            elif row['ORIGINAL JERSEY'] == 'Lunyov':
                name = 'Andrey Lunev'
            elif row['ORIGINAL JERSEY'] == 'Roennow':
                name = 'Frederik Rönnow'
            elif row['ORIGINAL JERSEY'] == 'I îgîrla':
                name = 'Igor Tigirlas'
            elif row['ORIGINAL JERSEY'] == 'T Alcantara':
                name = 'Thiago'
            elif row['ORIGINAL JERSEY'] == 'N`Zonzi':
                name = 'Steven Nzonzi'
            elif row['ORIGINAL JERSEY'] == 'Chu En-Le':
                name = 'Onur Dogan'
            elif row['ORIGINAL JERSEY'] == 'Um Serei Rath':
                name = 'Sereyroth Um'
            elif row['ORIGINAL JERSEY'] == 'A-nan P':
                name = 'Pokklaw Anan'
            elif row['ORIGINAL JERSEY'] == 'R Lásik':
                name = 'Richard Lasik'
            elif row['ORIGINAL JERSEY'] == 'Putera Nadher':
                name = 'Junior Eldstal'
            elif row['ORIGINAL JERSEY'] == 'D Łukasik':
                name = 'Daniel Lukasik'
            elif row['ORIGINAL JERSEY'] == 'P Ranelović':
                name = 'Predrag Randelovic'
            elif row['ORIGINAL JERSEY'] == 'Haroon Fakhruddin':
                name = 'Haroon Amiri'
            elif row['ORIGINAL JERSEY'] == 'Ala Addin Mahdi':
                name = 'Alaa Noman'
            elif row['ORIGINAL JERSEY'] == 'Ró-Ró':
                name = 'Pedro Miguel'
            elif row['ORIGINAL JERSEY'] == 'Tolo':
                name = 'Nouhou'
            elif row['ORIGINAL JERSEY'] == 'Zambo Anguissa André-Frank':
                name = 'Frank Anguissa'
            elif row['ORIGINAL JERSEY'] == 'Ati-Zigi':
                name = 'Lawrence Ati Zigi'
            elif row['ORIGINAL JERSEY'] == 'Bader Ahmed':
                name = 'Bader Al-Mutawa'
            elif row['ORIGINAL JERSEY'] == 'Elmutasem El Masrati':
                name = 'Al-Musrati'
            elif row['ORIGINAL JERSEY'] == 'Faisal Saleh':
                name = 'Faisal Al-Saleh'
            elif row['ORIGINAL JERSEY'] == 'N Tolo':
                name = 'Nouhou'
            elif row['ORIGINAL JERSEY'] == 'S Igboun':
                name = 'Sly'
            elif row['ORIGINAL JERSEY'] == 'D Dakonam':
                name = 'Djené'
            elif row['ORIGINAL JERSEY'] == 'N Mbaiam':
                name = 'Marius M\'Baiam'
            elif row['ORIGINAL JERSEY'] == 'Toni Cabaça':
                name = 'Tony'
            elif row['ORIGINAL JERSEY'] == 'M Khalil':
                name = 'Bessam Ahmed'
            elif row['ORIGINAL JERSEY'] == 'Yazc':
                name = 'Yusuf Yazıcı'
            elif row['ORIGINAL JERSEY'] == 'Nayr':
                name = 'Umut Nayir'
            elif row['ORIGINAL JERSEY'] == 'Daševičs':
                name = 'Eduards Daskevics'
            elif row['ORIGINAL JERSEY'] == 'S M\'Bia':
                name = 'Stéphane Mbia'
            elif row['ORIGINAL JERSEY'] == 'W Ylýasow':
                name = 'Vezirgeldy Ilyasov'
            elif row['ORIGINAL JERSEY'] == 'G Kwan Adsit':
                name = 'Gavin Kwan'
            elif row['ORIGINAL JERSEY'] == 'M Moqbel Sarhan':
                name = 'Mufeed Gamal'
            elif row['ORIGINAL JERSEY'] == 'M Ýakiýew':
                name = 'Murad Yakshyev'
            elif row['ORIGINAL JERSEY'] == 'Phm c Huy':
                name = 'Florentin Pham-Huy'
            elif row['ORIGINAL JERSEY'] == 'oàn Văn Hu':
                name = 'Van Hau Doan'
            elif row['ORIGINAL JERSEY'] == 'Kuehne':
                name = 'Simon Kühne'
            elif row['ORIGINAL JERSEY'] == 'Sadygov':
                name = 'Rashad Sadyqov'
            elif row['ORIGINAL JERSEY'] == 'Pacheta':
                name = 'Przemyslaw Placheta'
            elif row['ORIGINAL JERSEY'] == 'Costi':
                name = 'Giannis Kosti'
            elif row['ORIGINAL JERSEY'] == 'H Mejbri':
                name = 'Hannibal'
            elif row['ORIGINAL JERSEY'] == 'N El Shighail':
                name = 'Nasr Omar'
            elif row['ORIGINAL JERSEY'] == 'M Diaaeldin':
                name = 'Salaheldin Adil'
            elif row['ORIGINAL JERSEY'] == 'Yousuf Karah':
                name = 'Yousef Aijbali'
            elif row['ORIGINAL JERSEY'] == 'Anas Hamad':
                name = 'Anas Al-Misrati'
            elif row['ORIGINAL JERSEY'] == 'O Færø':
                name = 'Odmar Færø'
            elif row['ORIGINAL JERSEY'] == 'O Kļava':
                name = 'Oskars Klava'
            elif row['ORIGINAL JERSEY'] == 'Beto':
                name = 'Roberto'
            elif row['ORIGINAL JERSEY'] == 'W Pozo-Venta':
                name = 'Willian Pozo'
            elif row['ORIGINAL JERSEY'] == 'M Lumpungu':
                name = 'Moïse Bombito'
            elif row['ORIGINAL JERSEY'] == 'L Fanai':
                name = 'Lalrempuia Fanai'
            elif row['ORIGINAL JERSEY'] == 'J Zorrilla':
                name = 'Jonathan Cantillana'
            elif row['ORIGINAL JERSEY'] == 'Yuan Chen Xavier Tsan':
                name = 'Xavier Chen'
            elif row['ORIGINAL JERSEY'] == 'Y Kavalyow':
                name = 'Yuriy Kovalev'
            elif row['ORIGINAL JERSEY'] == 'Doualla':
                name = 'Viera Ellong'
            elif row['ORIGINAL JERSEY'] == 'Putsila':
                name = 'Anton Putilo'
            elif row['ORIGINAL JERSEY'] == 'Agyemang-Badu':
                name = 'Emmanuel Badu'
            elif row['ORIGINAL JERSEY'] == 'Pico':
                name = 'Roberto Lopes'
            elif row['ORIGINAL JERSEY'] == 'R Yusef':
                name = 'Ramadan Yesuf'
            elif row['ORIGINAL JERSEY'] == 'Bapasy':
                name = 'Pascal Razakanantenaina'
            elif row['ORIGINAL JERSEY'] == 'A Al Faz':
                name = 'Ali Mohamed'
            elif row['ORIGINAL JERSEY'] == 'Y Siad Isman':
                name = 'Yabe Siad'
            elif row['ORIGINAL JERSEY'] == 'S Wangjam':
                name = 'Suresh Singh'
            elif row['ORIGINAL JERSEY'] == 'A Nondi Obiero':
                name = 'Amos Nondi'
            elif row['ORIGINAL JERSEY'] == 'S Esah Nain':
                name = 'Shaiful Esah'
            elif row['ORIGINAL JERSEY'] == 'A Kouame':
                name = 'Ange Samuel'
            elif row['ORIGINAL JERSEY'] == 'Abdel Raouf Yagoub':
                name = 'Abdelaziz Yagoub'
            elif row['ORIGINAL JERSEY'] == 'Y Lee':
                name = 'Yong-jik Ri'
            elif row['ORIGINAL JERSEY'] == 'D Peiris':
                name = 'Dilip Kurukulasuriyage'
            elif row['ORIGINAL JERSEY'] == 'D Peiris':
                name = 'Dilip Kurukulasuriyage'
            elif row['ORIGINAL JERSEY'] == 'F Dafé':
                name = 'Falimery Ramanamahefa'
            elif row['ORIGINAL JERSEY'] == 'Shen Zigui':
                name = 'Will Donkin'
            elif row['ORIGINAL JERSEY'] == 'O Nhaca':
                name = 'Cigano'
                
                

            
                

            
            dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique() 
            best_match = threshold_player_match(name, dataset_nationality)
            if best_match[0] != ['No matches found even with the lowest threshold.']:
                if best_match[1] >= 80:
                    print(index, name, nationality, best_match)
                    correct_name = best_match[0][0]
                    number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                    print(index, full_num, number, reason)
                    if reason != 'Did Not Find in Curr Season, One Up, or One Down':
                        other_cases.at[index, 'Name(s) Found'] = correct_name
                        other_cases.at[index, 'Status New'] = 'Done'
                        other_cases.at[index, 'Market Value New'] = number
                        other_cases.at[index, 'Match Case'] = 'single'
                        correct_cases += 1
            
    else:
        done_cases += 1
correct_cases, cases, done_cases


(0, 106, 871)

In [ ]:
#AD HOC 
#Tecatito - Jesús Corona
#Natkho - Bibras Natcho'
#Hahgverdi - Höccat Haqverdi
#W Ballakow - Velmyrat Ballakov
#H Aliqulov - Khusniddin Alikulov
#H Tn Tài - Tan Tai Ho
#Nasser Garba - Abdoul Garba
#A Radosavljevič - Aleksandar Radosavljevic
#Karčemarskas - Zydrunas Karcemarskas
#M Islam Nazir - Nasirul Islam
#Fernández Toro - Roberto Fernández
#Yerokhin - Aleksandr Erokhin
#Lunyov - Andrey Lunev 
#Roennow - Frederik Rönnow
#I îgîrla - Igor Tigirlas
#T Alcantara - Thiago 
#N`Zonzi - Steven Nzonzi
#Chu En-Le - Onur Dogan
#Um Serei Rath - Sereyroth Um
#A-nan P - Pokklaw Anan
#R Lásik - Richard Lasik
#Putera Nadher - Junior Eldstal
#D Łukasik - Daniel Lukasik
#P Ranelović - Predrag Randelovic
#Haroon Fakhruddin - Haroon Amiri
#Ala Addin Mahdi - Alaa Noman
#Ró-Ró - Pedro Miguel
#Tolo - Nouhou
#Zambo Anguissa André-Frank - Frank Anguissa
#Ati-Zigi - Lawrence Ati Zigi
#Bader Ahmed - Bader Al-Mutawa
#Elmutasem El Masrati - Al-Musrati
#Faisal Saleh - Faisal Al-Saleh
#N Tolo - Nouhou
#S Igboun - Sly
#D Dakonam - Djené
#N Mbaiam - Marius M'Baiam
#Toni Cabaça - Tony
#M Khalil - Bessam Ahmed
#Yazc - Yusuf Yazıcı
#Nayr - Umut Nayir
#Daševičs - Eduards Daskevics
#S M'Bia - Stéphane Mbia
#W Ylýasow - Vezirgeldy Ilyasov
#G Kwan Adsit - Gavin Kwan
#M Moqbel Sarhan - Mufeed Gamal
#M Ýakiýew - Murad Yakshyev
#Phm c Huy - Florentin Pham-Huy
#oàn Văn Hu - Van Hau Doan
#Kuehne - Simon Kühne
#Sadygov - Rashad Sadyqov'
#Pacheta - Przemyslaw Placheta
#Costi - Giannis Kosti
#H Mejbri - Hannibal
#N El Shighail
#M Diaaeldin - Salaheldin Adil 
#Yousuf Karah - Yousef Aijbali
#Anas Hamad - Anas Al-Misrati
#O Færø - Odmar Færø
#O Kļava - Oskars Klava
#Beto - Roberto
#W Pozo-Venta - Willian Pozo
#M Lumpungu - Moïse Bombito
#L Fanai - Lalrempuia Fanai
#J Zorrilla - Jonathan Cantillana
#Yuan Chen Xavier Tsan - Xavier Chen
#Y Kavalyow - Yuriy Kovalev
#Doualla - Viera Ellong
#Putsila - Anton Putilo 
#Agyemang-Badu - Emmanuel Badu

#Chiquito - Juan Flores?


for index, row in other_cases.iterrows():
    if row['Match'] == 'Kazakhstan vs Iceland':
        other_cases.at[index, 'Nationality'] = 'Iceland'
        other_cases.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
other_cases.at[584, 'Name(s) Found'] = 'Hannes Thór Halldórsson'
other_cases.at[584, 'Market Value New'] = 400000
other_cases.at[584, 'Status New'] = 'Done'
other_cases.at[584, 'Status'] = 'SUCCESS'
other_cases.at[584, 'Match Case'] = 'single'

other_cases.at[585, 'Name(s) Found'] = 'Birkir Már Saevarsson'
other_cases.at[585, 'Market Value New'] = 500000
other_cases.at[585, 'Status New'] = 'Done'
other_cases.at[585, 'Status'] = 'SUCCESS'
other_cases.at[585, 'Match Case'] = 'single'

other_cases.at[586, 'Name(s) Found'] = 'Ragnar Sigurdsson'
other_cases.at[586, 'Market Value New'] = 4000000
other_cases.at[586, 'Status New'] = 'Done'
other_cases.at[586, 'Status'] = 'SUCCESS'
other_cases.at[586, 'Match Case'] = 'single'

other_cases.at[587, 'Name(s) Found'] = 'Ari Freyr Skúlason'
other_cases.at[587, 'Market Value New'] = 700000
other_cases.at[587, 'Status New'] = 'Done'
other_cases.at[587, 'Status'] = 'SUCCESS'
other_cases.at[587, 'Match Case'] = 'single'

other_cases.at[588, 'Name(s) Found'] = 'Birkir Bjarnason'
other_cases.at[588, 'Market Value New'] = 1000000
other_cases.at[588, 'Status New'] = 'Done'
other_cases.at[588, 'Status'] = 'SUCCESS'
other_cases.at[588, 'Match Case'] = 'single'

other_cases.at[589, 'Name(s) Found'] = 'Gylfi Sigurdsson'
other_cases.at[589, 'Market Value New'] = 11000000
other_cases.at[589, 'Status New'] = 'Done'
other_cases.at[589, 'Status'] = 'SUCCESS'
other_cases.at[589, 'Match Case'] = 'single'

other_cases.at[590, 'Name(s) Found'] = 'Kári Árnason'
other_cases.at[590, 'Market Value New'] = 600000
other_cases.at[590, 'Status New'] = 'Done'
other_cases.at[590, 'Status'] = 'SUCCESS'
other_cases.at[590, 'Match Case'] = 'single'

other_cases.at[591, 'Name(s) Found'] = 'Aron Gunnarsson'
other_cases.at[591, 'Market Value New'] = 3000000
other_cases.at[591, 'Status New'] = 'Done'
other_cases.at[591, 'Status'] = 'SUCCESS'
other_cases.at[591, 'Match Case'] = 'single'

other_cases.at[592, 'Name(s) Found'] = 'Andri Gudjohnsen'
other_cases.at[592, 'Market Value New'] = 3000000
other_cases.at[592, 'Status New'] = 'Done'
other_cases.at[592, 'Status'] = 'SUCCESS'
other_cases.at[592, 'Match Case'] = 'single'

other_cases.at[593, 'Name(s) Found'] = 'Jóhann Berg Gudmundsson'
other_cases.at[593, 'Market Value New'] = 2000000
other_cases.at[593, 'Status New'] = 'Done'
other_cases.at[593, 'Status'] = 'SUCCESS'
other_cases.at[593, 'Match Case'] = 'single'

other_cases.at[594, 'Name(s) Found'] = 'Kolbeinn Sigthórsson'
other_cases.at[594, 'Market Value New'] = 4500000
other_cases.at[594, 'Status New'] = 'Done'
other_cases.at[594, 'Status'] = 'SUCCESS'
other_cases.at[594, 'Match Case'] = 'single'

for index, row in combined_df.iterrows():
    if row['ORIGINAL JERSEY'] == 'Reinildo':
        combined_df.at[index, 'Name(s) Found'] = 'Reinildo Mandava'
        if row['Season'] == 2021:
            combined_df.at[index, 'Market Value'] = 7000000
        else:
            combined_df.at[index, 'Market Value'] = 2400000


combined_df[combined_df['ORIGINAL JERSEY'] == 'Reinildo']

other_cases.at[141, 'Name(s) Found'] = 'Cigano'
other_cases.at[141, 'Market Value New'] = 0
other_cases.at[141, 'Status New'] = 'Done'
other_cases.at[141, 'Status'] = 'SUCCESS'
other_cases.at[141, 'Match Case'] = 'single'
other_cases.at[275, 'Name(s) Found'] = 'Cigano'
other_cases.at[275, 'Market Value New'] = 0
other_cases.at[275, 'Status New'] = 'Done'
other_cases.at[275, 'Status'] = 'SUCCESS'
other_cases.at[275, 'Match Case'] = 'single'
other_cases.at[284, 'Name(s) Found'] = 'Cigano'
other_cases.at[284, 'Market Value New'] = 0
other_cases.at[284, 'Status New'] = 'Done'
other_cases.at[284, 'Status'] = 'SUCCESS'
other_cases.at[284, 'Match Case'] = 'single'


In [217]:
other_cases.to_csv('other_cases_edited_may_8.csv')

In [216]:
other_cases[other_cases['Status New'] != 'Done']#['ORIGINAL JERSEY'].value_counts().head(20)

,Match ID,Competition,Date,Match,Season,Nationality,Country Code,ORIGINAL JERSEY,Lookup Still Required?,Lookup Return Case,Name(s) Found,Translated Name,Original Case,To Remove,Status New,Market Value New,Match Case,Status
0,2681,"World Cup Qualifiers, Asia, 2022",03-06-2021,Thailand vs Indonesia,2021,Thailand,TH,E Amantegui,True,Lookup Required,0,NaN,Match Date Filter Said 0,False,NaN,NaN,NaN,NaN
1,2681,"World Cup Qualifiers, Asia, 2022",03-06-2021,Thailand vs Indonesia,2021,Indonesia,ID,W Sulaeman,True,Lookup Required,0,NaN,Match Date Filter Said 0,False,NaN,NaN,NaN,NaN
22,2707,"World Cup Qualifiers, Asia, 2026",17-10-2023,Brunei vs Indonesia,2023,Indonesia,ID,W Sulaeman,True,Lookup Required,0,NaN,Match Date Filter Said 0,False,NaN,NaN,NaN,NaN
23,2711,"World Cup Qualifiers, Asia, 2026",16-11-2023,Oman vs Chinese Taipei,2023,Chinese Taipei,TW,Chin Wen-yen,True,Lookup Required,0,NaN,Match Date Filter Said 0,False,NaN,NaN,NaN,NaN
29,2727,"World Cup Qualifiers, Asia, 2026",16-11-2023,Saudi Arabia vs Pakistan,2023,Saudi Arabia,SA,Muath Faqeehi,True,Lookup Required,0,NaN,Match Date Filter Said 0,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,2150,"World Cup Qualifiers, Africa, 2022",02-09-2021,Congo DR vs Tanzania,2021,Tanzania,TZ,R Mwakasugule,True,Lookup Required,0,NaN,Match Date Filter Said 0,False,NaN,NaN,NaN,NaN
882,2157,"World Cup Qualifiers, Africa, 2022",05-09-2019,Somalia vs Zimbabwe,2019,Somalia,SO,F Abdisamed Ismael,True,Lookup Required,0,NaN,Match Date Filter Said 0,False,NaN,NaN,NaN,NaN
969,1467,Nations League 2018-2020,09.09.18,Cyprus vs Slovenia,2018,Cyprus,CY,Demetriou,True,Lookup Required,0,NaN,Match Date Filter Said 0,False,NaN,NaN,NaN,NaN
970,1469,Nations League 2018-2020,13.10.18,Bulgaria vs Cyprus,2018,Cyprus,CY,Demetriou,True,Lookup Required,0,NaN,Match Date Filter Said 0,False,NaN,NaN,NaN,NaN


In [230]:
other_cases['Nationality'].value_counts().head(30)

Nationality
Martinique                          122
Brunei Darussalam                    66
China                                56
Guadeloupe                           43
French Guiana                        33
Eritrea                              31
Cameroon                             28
Madagascar                           23
Israel                               23
Libya                                21
Faroe Islands                        19
Chinese Taipei                       18
Cyprus                               17
Sudan                                16
Hong Kong                            16
Togo                                 16
Niger                                15
Thailand                             15
Vietnam                              15
Belarus                              14
Democratic Republic of the Congo     14
Ethiopia                             14
Turkey                               14
Azerbaijan                           13
Yemen                       

In [31]:
other_cases[other_cases['Nationality'] == namey]['ORIGINAL JERSEY'].value_counts()

ORIGINAL JERSEY
J Kasusula    6
B Bompunga    2
M Amale       1
M Ndonga      1
Z Matumona    1
E Bokanga     1
L Mbele       1
N Idumba      1
Name: count, dtype: int64

In [29]:
namey = 'Democratic Republic of the Congo'
leagues_value[leagues_value['Nationality'] == namey]['Season'].value_counts()

Season
2019    74
2020    65
2022    63
2021    62
2015    58
2016    54
2017    51
2014    50
2011    46
2018    27
2023    25
2012    23
Name: count, dtype: int64

In [30]:
leagues_value_large[leagues_value_large["Nationality"] == namey]['Season'].value_counts()

Season
2019    74
2020    65
2022    63
2021    62
2015    58
2016    54
2017    51
2014    50
2018    27
2023    24
Name: count, dtype: int64

In [226]:
other_cases.to_csv('other_cases_wrong_edited.csv')

### OTHER CASES - FINISHING

In [236]:
other_cases_edited = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Red Groups/other_cases_edited_may_8.csv')

In [237]:
correct_cases = 0
fuck_me = 0
for index, row in other_cases_edited.iterrows():
    if row['Match Case'] == 'single' and row['Status New'] != 'Done':
        name = row['Name(s) Found']
        full_num = row['Season']
        country_code_this = row['Country Code']

        number, reason = find_in_transfermarkt(name, full_num, country_code_this, leagues_value)

        #print(index, full_num, number, reason)
        if reason != 'Did Not Find in Curr Season, One Up, or One Down':
            other_cases_edited.at[index, 'Name(s) Found'] = correct_name
            other_cases_edited.at[index, 'Status New'] = 'Done'
            other_cases_edited.at[index, 'Market Value New'] = number
            other_cases_edited.at[index, 'Match Case'] = 'single'
            correct_cases += 1
        else:
            print(index)
            fuck_me += 1
correct_cases, fuck_me

0 1 0
1 1 1
1 1 0
0 0 0
23
0 0 0
29
1 1 0
Ivan Wani 0 Only Before X
1 0 0
Henok Adugna -1 Both Empty
0 0 0
40
1 1 0
Junior Noubi 90000.0 Only Before
1 1 1
0 0 0
51
1 1 1
1 1 1
Jean Monuma 0 Had Both but Both Were Empty
1 1 1
Jean Monuma 0 Had Both but Both Were Empty
1 1 1
Brandón de León 0 Had Both but Both Were Empty
1 1 1
Brandón de León 0 Had Both but Both Were Empty
0 1 0
1 1 0
Guiguiban Loubandem 0 Only Before X
1 1 0
Mustaf Yuusuf 0 Only Before X
1 0 0
Henok Adugna -1 Both Empty
0 0 0
146
0 0 0
147
0 0 0
149
0 0 0
150
1 1 0
Ivan Wani 0 Only Before X
1 1 0
Yabe Siad 0 Only Before X
1 1 0
Guiguiban Loubandem 0 Only Before X
0 0 0
169
0 1 0
0 1 1
0 1 1
1 1 1
0 0 0
226
0 1 0
0 0 0
237
1 1 0
0 1 0
1 1 1
Mohamed Saeed Ahmed 0 Had Both but Both Were Empty
0 1 0
1 0 1
Eduardo Varela 0 Only After X
1 0 1
Vavá Pequeno 0 Only After X
1 1 1
Iniesta 0 Had Both but Both Were Empty
1 0 1
Reliants Lusajo 0 Only After X
0 1 0
0 1 0
1 1 0
1 1 1
0 0 0
314
1 1 0
Izan Fernández 0 Only Before X
1 1 0

(80, 22)

In [240]:
leagues_value_large[leagues_value_large['Name'] == 'Wen-Yen Chin']

,Unnamed: 0,Name,Nationality,Position,Market Value,Season,League,Team,Team 1 Code,Birthday,Age,PlayerID,Accents Removed


In [241]:
other_cases_edited.at[23, 'Market Value New'] = 50000
other_cases_edited.at[23, 'Status New'] = 'Done'

other_cases_edited.at[29, 'Market Value New'] = 500000
other_cases_edited.at[29, 'Status New'] = 'Done'

other_cases_edited.at[40, 'Market Value New'] = 800000
other_cases_edited.at[40, 'Status New'] = 'Done'
other_cases_edited.at[51, 'Status New'] = 'Done'

other_cases_edited.at[51, 'Market Value New'] = 0
other_cases_edited.at[146, 'Status New'] = 'Done'

other_cases_edited.at[146, 'Market Value New'] = 0
other_cases_edited.at[147, 'Status New'] = 'Done'

other_cases_edited.at[147, 'Market Value New'] = 0

other_cases_edited.at[149, 'Status New'] = 'Done'

other_cases_edited.at[149, 'Market Value New'] = 0

other_cases_edited.at[150, 'Status New'] = 'Done'

other_cases_edited.at[150, 'Market Value New'] = 0

other_cases_edited.at[169, 'Status New'] = 'Done'

other_cases_edited.at[169, 'Market Value New'] = 200000

other_cases_edited.at[226, 'Status New'] = 'Done'

other_cases_edited.at[226, 'Market Value New'] = 0

other_cases_edited.at[237, 'Status New'] = 'Done'

other_cases_edited.at[237, 'Market Value New'] = 0

other_cases_edited.at[314, 'Status New'] = 'Done'

other_cases_edited.at[314, 'Market Value New'] = 0

other_cases_edited.at[348, 'Status New'] = 'Done'

other_cases_edited.at[348, 'Market Value New'] = 0

other_cases_edited.at[472, 'Status New'] = 'Done'

other_cases_edited.at[472, 'Market Value New'] = 0

other_cases_edited.at[473, 'Status New'] = 'Done'

other_cases_edited.at[473, 'Market Value New'] = 0

other_cases_edited.at[474, 'Status New'] = 'Done'

other_cases_edited.at[474, 'Market Value New'] = 0

other_cases_edited.at[478, 'Status New'] = 'Done'

other_cases_edited.at[478, 'Market Value New'] = 200000

other_cases_edited.at[479, 'Status New'] = 'Done'

other_cases_edited.at[479, 'Market Value New'] = 0

other_cases_edited.at[480, 'Status New'] = 'Done'

other_cases_edited.at[480, 'Market Value New'] = 0

other_cases_edited.at[481, 'Status New'] = 'Done'

other_cases_edited.at[481, 'Market Value New'] = 0

other_cases_edited.at[611, 'Status New'] = 'Done'

other_cases_edited.at[611, 'Market Value New'] = 0

other_cases_edited.at[787, 'Status New'] = 'Done'

other_cases_edited.at[787, 'Market Value New'] = 0


In [243]:
other_cases_edited['Status New'].value_counts()

Status New
Done    972
Name: count, dtype: int64

In [244]:
other_cases_edited.to_csv('other_cases_DONE.csv')

## failed_finding_season_up_down

In [273]:
combined_df_may_10 = load_csv_dataset('combined_df_may_10.csv')
combined_df_may_10 = combined_df_may_10.drop(columns={'Unnamed: 0'})

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_49517/1460689172.py:54: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [332]:
combined_df_may_10_terrikon_removed = combined_df_may_10[~combined_df_may_10['Competition'].isin(['Euros 2016, Qualifiers', 'Euros 2020, Qualifiers', 'Euros 2024, Qualifiers', 
                                                                           'Nations League 2018-2020', 'Nations League 2020-2021', 'Nations League 2022-2023', 
                                                                           'World Cup 2022', 'World Cup 2018', 'World Cup 2014', 
                                                                           'Euros 2016', 'Euros 2020', 
                                                                           'Copa America 2021', 'Copa America 2015', 'Copa America 2019'])]
combined_df_may_10_terrikon_removed

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
0,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Anthony Moris,400000.0,False,working,False,Anthony Moris,A Moris,single
1,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Laurent Jans,1200000.0,False,working,False,Laurent Jans,L Jans,single
2,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Kevin Malget,150000.0,False,working,False,Kevin Malget,K Malget,single
3,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Mario Mutsch,125000.0,False,working,False,Mario Mutsch,M Mutsch,single
4,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Chris Philipps,1250000.0,False,working,False,Chris Philipps,C Philipps,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90753,1343,"Olympics, 2016",Germany vs South Korea,2016-08-07,South Korea,KR,2016,SUCCESS,Kyu-baek Choi,350000.0,False,Found in Curr Season,False,Kyu-baek Choi,Choi Kyu-baek,single
90754,1375,"Olympics, 2020",Egypt vs Argentina,2021-07-25,Argentina,AR,2021,SUCCESS,Claudio Nicolás Bravo,2000000.0,False,Found in Curr Season,False,Claudio Nicolás Bravo,Claudio Bravo,single
90755,1378,"Olympics, 2020",Spain vs Argentina,2021-07-28,Argentina,AR,2021,SUCCESS,Claudio Nicolás Bravo,2000000.0,False,Found in Curr Season,False,Claudio Nicolás Bravo,Claudio Bravo,single
90756,1377,"Olympics, 2020",Australia vs Egypt,2021-07-28,Australia,AU,2021,SUCCESS,Dylan Pierias,350000.0,False,Found in Curr Season,False,Dylan Pierias,Dylan Pierias,single


In [336]:
failed_all_seasons = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Red Groups/failed_finding_season_up_down.csv')
failed_all_seasons = failed_all_seasons.drop(columns={'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'})

failed_all_seasons_terrikon_removed = failed_all_seasons[~failed_all_seasons['Competition'].isin(['Euros 2016, Qualifiers', 'Euros 2020, Qualifiers', 'Euros 2024, Qualifiers', 
                                                                           'Nations League 2018-2020', 'Nations League 2020-2021', 'Nations League 2022-2023', 
                                                                           'World Cup 2022', 'World Cup 2018', 'World Cup 2014', 
                                                                           'Euros 2016', 'Euros 2020', 
                                                                           'Copa America 2021', 'Copa America 2015', 'Copa America 2019'])]
failed_all_seasons_terrikon_removed

,Match ID,Competition,Date,Match,Season,Nationality,Country Code,ORIGINAL JERSEY,Lookup Still Required?,Lookup Return Case,Name(s) Found,Translated Name,Original Case,To Remove
69,2588,"World Cup Qualifiers, Asia, 2022",03-06-2021,Bangladesh vs Afghanistan,2021,Bangladesh,BD,M Ra,True,Failed Using Name Looking at Season and Season...,['Monaem Raju Khan'],NaN,Group 1 Fail,False
70,2592,"World Cup Qualifiers, Asia, 2022",07-06-2021,Bangladesh vs India,2021,Bangladesh,BD,M Ra,True,Failed Using Name Looking at Season and Season...,['Monaem Raju Khan'],NaN,Group 1 Fail,False
71,2611,"World Cup Qualifiers, Asia, 2022",5-9-2019,Indonesia vs Malaysia,2019,Indonesia,ID,Zul,True,Failed Using Name Looking at Season and Season...,['Zulham Zamrun'],NaN,Group 1 Fail,False
72,2623,"World Cup Qualifiers, Asia, 2022",10-10-2019,Bangladesh vs Qatar,2019,Bangladesh,BD,M Ra,True,Failed Using Name Looking at Season and Season...,['Monaem Raju Khan'],NaN,Group 1 Fail,False
77,1935,"World Cup Qualifiers, Africa, 2018",12-11-2017,Ghana vs Egypt,2017,Ghana,GH,E Gyasi,True,Failed Using Name Looking at Season and Season...,['Emmanuel Gyasi'],NaN,Group 1 Fail,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,2161,"World Cup Qualifiers, Africa, 2022",01-09-2021,Senegal vs Togo,2021,Togo,TG,J Nadjombe,True,Failed Using Name Looking at Season and Season...,Jean-Marie Nadjombe,NaN,Match Date Filter Said 0,False
659,2166,"World Cup Qualifiers, Africa, 2022",15-11-2021,Guinea-Bissau vs Sudan,2021,Sudan,SD,F Mamoun,True,Failed Using Name Looking at Season and Season...,Omer Mahmoud,NaN,Match Date Filter Said 0,False
660,2169,"World Cup Qualifiers, Africa, 2022",06-09-2021,South Africa vs Ghana,2021,Ghana,GH,F Mohammed,True,Failed Using Name Looking at Season and Season...,Fatawu Mohammed,NaN,Match Date Filter Said 0,False
661,3721,"Gold Cup, 2019",16-06-2019,Mexico vs Cuba,2019,Cuba,CU,K Aldair,True,Failed Using Name Looking at Season and Season...,Aldair Ruiz,NaN,Match Date Filter Said 0,False


In [340]:
combined_df_may_10_terrikon_removed.to_csv('combined_DF_terrikon_removed.csv')

In [341]:
failed_all_seasons_terrikon_removed.to_csv('failed_df_terrikon_removed.csv')

## May 14

In [353]:
combined_df_may_14 = load_csv_dataset('combined_df_may_14_updated.csv')
len(combined_df_may_14)

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_49517/1460689172.py:54: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


92172

In [350]:
failed_group_terrikon_removed = load_csv_dataset('May 13 latest data - removed terrikon - downloaded new TM lineups/failed_df_terrikon_removed_IDs_FIXED.csv')

In [402]:
# failed_group_terrikon_removed.at[109, 'Name(s) Found'] = 'Mohamud Ali'
# failed_group_terrikon_removed.at[109, 'Translated Name'] = 'Mohamud Ali'

In [366]:
# for index, row in failed_group_terrikon_removed.iterrows():
#     if failed_group_terrikon_removed.at[index, 'Name(s) Found'].startswith('['):
#         edited_name = eval(failed_group_terrikon_removed.at[index, 'Name(s) Found'])[0]
#         failed_group_terrikon_removed.at[index, 'Name(s) Found'] = edited_name
#         #print(index, combined_df.at[index, 'Name(s) Found'])

# failed_group_terrikon_removed[failed_group_terrikon_removed['Name(s) Found'].str.startswith('[')]

failed_group_terrikon_removed

,Unnamed: 0.1,Unnamed: 0,Competition,Date,Match,Season,Nationality,Country Code,ORIGINAL JERSEY,Lookup Still Required?,Lookup Return Case,Name(s) Found,Translated Name,Original Case,To Remove,Match ID,Competition General
0,0,69,"World Cup Qualifiers, Asia, 2022",03-06-2021,Bangladesh vs Afghanistan,2021,Bangladesh,BD,M Ra,True,Failed Using Name Looking at Season and Season...,Monaem Raju Khan,NaN,Group 1 Fail,False,"World Cup Qualifiers, Asia_953","World Cup Qualifiers, Asia"
1,1,70,"World Cup Qualifiers, Asia, 2022",07-06-2021,Bangladesh vs India,2021,Bangladesh,BD,M Ra,True,Failed Using Name Looking at Season and Season...,Monaem Raju Khan,NaN,Group 1 Fail,False,"World Cup Qualifiers, Asia_957","World Cup Qualifiers, Asia"
2,2,71,"World Cup Qualifiers, Asia, 2022",5-9-2019,Indonesia vs Malaysia,2019,Indonesia,ID,Zul,True,Failed Using Name Looking at Season and Season...,Zulham Zamrun,NaN,Group 1 Fail,False,"World Cup Qualifiers, Asia_976","World Cup Qualifiers, Asia"
3,3,72,"World Cup Qualifiers, Asia, 2022",10-10-2019,Bangladesh vs Qatar,2019,Bangladesh,BD,M Ra,True,Failed Using Name Looking at Season and Season...,Monaem Raju Khan,NaN,Group 1 Fail,False,"World Cup Qualifiers, Asia_988","World Cup Qualifiers, Asia"
4,4,77,"World Cup Qualifiers, Africa, 2018",12-11-2017,Ghana vs Egypt,2017,Ghana,GH,E Gyasi,True,Failed Using Name Looking at Season and Season...,Emmanuel Gyasi,NaN,Group 1 Fail,False,"World Cup Qualifiers, Africa_1367","World Cup Qualifiers, Africa"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,425,658,"World Cup Qualifiers, Africa, 2022",01-09-2021,Senegal vs Togo,2021,Togo,TG,J Nadjombe,True,Failed Using Name Looking at Season and Season...,Jean-Marie Nadjombe,NaN,Match Date Filter Said 0,False,"World Cup Qualifiers, Africa_1574","World Cup Qualifiers, Africa"
423,426,659,"World Cup Qualifiers, Africa, 2022",15-11-2021,Guinea-Bissau vs Sudan,2021,Sudan,SD,F Mamoun,True,Failed Using Name Looking at Season and Season...,Omer Mahmoud,NaN,Match Date Filter Said 0,False,"World Cup Qualifiers, Africa_1578","World Cup Qualifiers, Africa"
424,427,660,"World Cup Qualifiers, Africa, 2022",06-09-2021,South Africa vs Ghana,2021,Ghana,GH,F Mohammed,True,Failed Using Name Looking at Season and Season...,Fatawu Mohammed,NaN,Match Date Filter Said 0,False,"World Cup Qualifiers, Africa_1581","World Cup Qualifiers, Africa"
425,428,661,"Gold Cup, 2019",16-06-2019,Mexico vs Cuba,2019,Cuba,CU,K Aldair,True,Failed Using Name Looking at Season and Season...,Aldair Ruiz,NaN,Match Date Filter Said 0,False,Gold Cup_1908,Gold Cup


In [435]:
pd.set_option('display.max_rows', 50)


In [438]:
combined_df_may_14[combined_df_may_14['ORIGINAL JERSEY'] == 'A Kamara']

,Unnamed: 0.1,Unnamed: 0,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General,Match ID
8210,8210,8574,"World Cup Qualifiers, Africa, 2022",Cape Verde vs Liberia,10-10-2021,Liberia,LR,2021,SUCCESS,Abu Kamara,75000.0,False,working,False,Abu Kamara,A Kamara,single,"World Cup Qualifiers, Africa","World Cup Qualifiers, Africa_1308"
27959,27959,28455,"World Cup Qualifiers, Africa, 2022",Central African Republic vs Liberia,06-09-2021,Liberia,LR,2021,SUCCESS,Abu Kamara,75000.0,False,working,False,Abu Kamara,A Kamara,single,"World Cup Qualifiers, Africa","World Cup Qualifiers, Africa_1556"
28298,28298,28794,"World Cup Qualifiers, Africa, 2022",Liberia vs Nigeria,13-11-2021,Liberia,LR,2021,SUCCESS,Abu Kamara,75000.0,False,working,False,Abu Kamara,A Kamara,single,"World Cup Qualifiers, Africa","World Cup Qualifiers, Africa_1582"
38562,38562,39171,"World Cup Qualifiers, Africa, 2018",Chad vs Sierra Leone,10-10-2015,Sierra Leone,SL,2015,SUCCESS,Alhaji Kamara,350000.0,False,Found in Curr Season,True,Alhaji Kamara,A Kamara,single,"World Cup Qualifiers, Africa","World Cup Qualifiers, Africa_1504"
38841,38841,39451,"African Cup of Nations, 2021",Algeria vs Sierra Leone,11-01-2022,Sierra Leone,SL,2022,SUCCESS,Alhaji Kamara,700000.0,False,Found in Curr Season,False,Alhaji Kamara,A Kamara,single,African Cup of Nations,African Cup of Nations_1695
38861,38861,39471,"African Cup of Nations, 2021",Mauritania vs Gambia,12-01-2022,Mauritania,MR,2022,SUCCESS,Aboubakar Kamara,3500000.0,False,Found in Curr Season,False,Aboubakar Kamara,A Kamara,single,African Cup of Nations,African Cup of Nations_1710
38865,38865,39475,"African Cup of Nations, 2021",Tunisia vs Mauritania,16-01-2022,Mauritania,MR,2022,SUCCESS,Aboubakar Kamara,3500000.0,False,Found in Curr Season,False,Aboubakar Kamara,A Kamara,single,African Cup of Nations,African Cup of Nations_1714
39124,39124,39785,"World Cup Qualifiers, Africa, 2026",Ethiopia vs Sierra Leone,15-11-2023,Sierra Leone,SL,2023,SUCCESS,Alhaji Kamara,300000.0,False,Found in Curr Season,False,Alhaji Kamara,A Kamara,single,"World Cup Qualifiers, Africa","World Cup Qualifiers, Africa_1289"
39212,39212,39894,"World Cup Qualifiers, Africa, 2022",Mauritania vs Equatorial Guinea,16-11-2021,Mauritania,MR,2021,SUCCESS,['Aboubakar Kamara'],2000000.0,False,Found in Curr Season,False,Aboubakar Kamara,A Kamara,single,"World Cup Qualifiers, Africa","World Cup Qualifiers, Africa_1560"
39213,39213,39895,"World Cup Qualifiers, Africa, 2022",Zambia vs Mauritania,13-11-2021,Mauritania,MR,2021,SUCCESS,['Aboubakar Kamara'],2000000.0,False,Found in Curr Season,False,Aboubakar Kamara,A Kamara,single,"World Cup Qualifiers, Africa","World Cup Qualifiers, Africa_1564"


In [436]:
combined_df_may_14[(combined_df_may_14['Competition'] == 'Gold Cup, 2023')]#.head(500)#& (combined_df_may_14['Lookup Return Case'] == 'Had Both but Both Were Empty')#['Lookup Return Case'].value_counts()

,Unnamed: 0.1,Unnamed: 0,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General,Match ID
6362,6362,6726,"Gold Cup, 2023",Qatar vs Honduras,30-06-2023,Qatar,QA,2023,SUCCESS,Meshaal Barsham,550000.0,False,working,False,Meshaal Barsham,Meshaal Barsham,single,Gold Cup,Gold Cup_1807
6363,6363,6727,"Gold Cup, 2023",Qatar vs Honduras,30-06-2023,Qatar,QA,2023,SUCCESS,Bassam Al-Rawi,600000.0,False,working,False,Bassam Al-Rawi,Bassam Al Rawi,single,Gold Cup,Gold Cup_1807
6364,6364,6728,"Gold Cup, 2023",Qatar vs Honduras,30-06-2023,Qatar,QA,2023,SUCCESS,Tarek Salman,450000.0,False,working,False,Tarek Salman,Tarek Salman,single,Gold Cup,Gold Cup_1807
6365,6365,6729,"Gold Cup, 2023",Qatar vs Honduras,30-06-2023,Qatar,QA,2023,SUCCESS,Ahmed Fathi,325000.0,False,working,False,Ahmed Fathi,Ahmed Fathi,single,Gold Cup,Gold Cup_1807
6366,6366,6730,"Gold Cup, 2023",Qatar vs Honduras,30-06-2023,Qatar,QA,2023,SUCCESS,Yusuf Abdurisag,300000.0,False,working,False,Yusuf Abdurisag,Yusuf Abdurisag,single,Gold Cup,Gold Cup_1807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56562,56562,57262,"Gold Cup, 2023",Canada vs Guadeloupe,28-06-2023,Canada,CA,2023,SUCCESS,Moïse Bombito,0.0,True,Found in Curr Season,False,Moïse Bombito,M Lumpungu,single,Gold Cup,Gold Cup_1876
56563,56563,57263,"Gold Cup, 2023",Canada vs Guadeloupe,28-06-2023,Guadeloupe,GP,2023,FAIL - NO DATA,0,NaN,True,Lookup Required,False,0,M Lina,none,Gold Cup,Gold Cup_1876
56564,56564,57264,"Gold Cup, 2023",Canada vs Guadeloupe,28-06-2023,Guadeloupe,GP,2023,FAIL - NO DATA,0,NaN,True,Lookup Required,False,0,A Baron,none,Gold Cup,Gold Cup_1876
56565,56565,57265,"Gold Cup, 2023",Canada vs Guadeloupe,28-06-2023,Guadeloupe,GP,2023,FAIL - NO DATA,0,NaN,True,Lookup Required,False,0,J Rotsen,none,Gold Cup,Gold Cup_1876


In [439]:
single_case_counter = 0
for index, row in failed_group_terrikon_removed.iterrows():
    this_jersey = row['ORIGINAL JERSEY']
    nationality = row['Nationality']

    names_found_combined_Df = combined_df_may_14[(combined_df_may_14['ORIGINAL JERSEY'] == this_jersey) & (combined_df_may_14['Nationality'] == nationality)]['Name(s) Found'].unique()
    
    if len(names_found_combined_Df) == 0:
        0==0 #these are the people not in the DF yet.
    elif len(names_found_combined_Df) == 1:
        print(index, this_jersey, (names_found_combined_Df), row['Nationality'])
        single_case_counter += 1
    
single_case_counter

    
    #

5 A Diaby ['Alassane Diaby'] Mali
6 A Diaby ['Alassane Diaby'] Mali
7 A Diaby ['Alassane Diaby'] Mali
16 H Ojamaa ['Hindrek Ojamaa'] Estonia
17 A Struna ['Aljaz Struna'] Slovenia
19 H Ojamaa ['Hindrek Ojamaa'] Estonia
20 A Struna ['Aljaz Struna'] Slovenia
24 A Diaby ['Alassane Diaby'] Mali
30 R Sadqov ['Rashad Sadiqov'] Azerbaijan
31 H Anier ['Hannes Anier'] Estonia
32 H Anier ['Hannes Anier'] Estonia
33 B Halimi ['Betim Halimi'] Kosovo
43 H Ojamaa ['Hindrek Ojamaa'] Estonia
46 A Struna ['Aljaz Struna'] Slovenia
52 H Anier ['Hannes Anier'] Estonia
53 H Anier ['Hannes Anier'] Estonia
54 H Anier ['Hannes Anier'] Estonia
55 R Sadqov ['Rashad Sadiqov'] Azerbaijan
57 H Anier ['Hannes Anier'] Estonia
58 B Halimi ['Betim Halimi'] Kosovo
59 Y Butt ['Yaqoob Butt'] Pakistan
60 Y Butt ['Yaqoob Butt'] Pakistan
62 H Anier ['Hannes Anier'] Estonia
70 Mohamed Fasal ['Fasal Naizer'] Sri Lanka
76 H Anier ['Hannes Anier'] Estonia
77 H Anier ['Hannes Anier'] Estonia
78 H Anier ['Hannes Anier'] Estonia
80

154

In [ ]:
#last group - manual fixes 

#282, 284, 325 and 330 - Ucha Lobjanidze

#343 and 408 name is Resul Çaryýew
#17, 20 and 46 is Andraz Struna

#209, 300, 302, 363, 364 - this is Mohamed Koffi
#301, 333, 352 - Amadou Wonkoye

#359 - Abdoukader Djama
#16, 19 and 43 - Henrik Ojamaa



#COMBINED DF FIXES 
#1541 and 5982 is Resul Çaryýew
#27800 is Amadou Wonkoye
#31194 is Mohamed Koffi
#35166 is Abdoukader Djama
#9700 and 25072 - Ucha Lobjanidze
#38932 and 39277 - Henrik Ojamaa


### Splitting into groups - OLD

In [266]:
match_date_filter_0_group = failed_all_seasons[failed_all_seasons['Original Case'] == 'Match Date Filter Said 0']
match_date_filter_0_group.to_csv('fail_group_match_date_filter_0.csv')

In [267]:
group_1_fails = failed_all_seasons[failed_all_seasons["Original Case"] == 'Group 1 Fail']
group_1_fails.to_csv('fail_group_group_1_fails.csv')

In [268]:
failed_other_groups = failed_all_seasons[(failed_all_seasons['Original Case'] != 'Match Date Filter Said 0') & (failed_all_seasons['Original Case'] != 'Group 1 Fail')]
failed_other_groups.to_csv('failed_group_others.csv')

#### Match Date filter 0 grup

In [ ]:
for index, row in combined_df_may_10.iterrows():
    if combined_df_may_10.at[index, 'Name(s) Found'].startswith('['):
        edited_name = eval(combined_df_may_10.at[index, 'Name(s) Found'])[0]
        combined_df_may_10.at[index, 'Name(s) Found'] = edited_name
        #print(index, combined_df.at[index, 'Name(s) Found'])

combined_df_may_10[combined_df_may_10['Name(s) Found'].str.startswith('[')]

In [286]:
for index, row in combined_df_may_10.iterrows():
    if row['ORIGINAL JERSEY'] == 'F Mamoun':
        combined_df_may_10.at[index, 'Name(s) Found'] = 'Faris Abdalla'
        combined_df_may_10.at[index, 'Name'] = 'Faris Abdalla'
    if row['ORIGINAL JERSEY'] == 'Jn Williams':
        combined_df_may_10.at[index, 'Name(s) Found'] = 'Jonathan Williams'
        combined_df_may_10.at[index, 'Name'] = 'Jonathan Williams'


In [291]:
combined_df_may_10.at[47240, 'Market Value'] = 750000
combined_df_may_10.at[61520, 'Market Value'] = 1000000


In [271]:
failed_group_match_date_filter = load_csv_dataset('fail_group_match_date_filter_0.csv')


#this group



#check if this jersey has 1 player with it in the combined dF 

In [285]:
single_case_counter = 0
for index, row in failed_group_match_date_filter.iterrows():
    this_jersey = row['ORIGINAL JERSEY']

    names_found_combined_Df = combined_df_may_10[combined_df_may_10['ORIGINAL JERSEY'] == this_jersey]['Name(s) Found'].unique()
    
    if len(names_found_combined_Df) == 0:
        0==0 #these are the people not in the DF yet.
    elif len(names_found_combined_Df) == 1:
        print(index, this_jersey, (names_found_combined_Df), row['Nationality'])
        single_case_counter += 1
    
single_case_counter

    

39 A Fatawu ['Hamidu Abdul Fatawu'] Ghana
42 V Aleksanyan ['Valeri Aleksanyan'] Armenia
45 Mohamed Fasal ['Fasal Naizer'] Sri Lanka
57 Y sako ['Yuya Osako'] Japan
58 Mohsin Al Khaldi ['Mohsin Jouhar'] Oman
59 Ouk Sovann ['Sovann Ouk'] Cambodia
60 Fahad Al Rashidi ['Fahad Aidh Al-Rashidi'] Kuwait
68 Mohammed Osman ['Mohammed Osman'] Syria
73 Omid Popalzay ['Omid Popalzay'] Afghanistan
74 Najim Haidary ['Najim Haidary'] Afghanistan
75 Noor Husin ['Noor Husin'] Afghanistan
76 Omran Haydary ['Omran Haydary'] Afghanistan
77 Jabar Sharza ['Jabar Sharza'] Afghanistan
82 A Williams ['Ashley Williams'] Liberia
88 M Ngele ['Mogakolodi Ngele'] Botswana
89 Mohammad Ali ['Mohammad Ali'] Jordan
91 Mohammad Ali ['Mohammad Ali'] Jordan
92 Mohammad Ali ['Mohammad Ali'] Jordan
96 M Ko ['Mbah Koné'] Burkina Faso
104 Rami Rabia ['Rami Rabia'] Egypt
106 Fábio Abreu ['Fabio Abreu'] Angola
115 L Ralte ['Lalrindika Ralte'] India
118 T Do ['Tristan Do'] Thailand
119 N Selanon ['Nitipong Selanon'] Thailand
122 

124